# Breaching privacy

This notebook does the same job as the cmd-line tool `breach.py`, but also directly visualizes the user data and reconstruction

In [ ]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

In [ ]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=['attack=imprint', 'case/server=malicious-model',
                                                      'case/user=local_updates'])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [ ]:
cfg.case.user.num_local_updates = 8
cfg.case.user.num_data_per_local_update_step = 8
cfg.case.user.num_data_points = 64
cfg.case.user.local_learning_rate = 1e-4

cfg.case.server.model_modification.type = 'ImprintBlock' 
cfg.case.server.model_modification.num_bins=128
cfg.case.server.model_modification.position=None
cfg.case.server.normalize_rounds=0


cfg.case.server.has_external_data = False

### Instantiate all parties

In [ ]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [ ]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

In [ ]:
print(f'Grad norm: {torch.norm(torch.stack([torch.norm(g, 2) for g in shared_data["gradients"][0]]), 2)}')
[(g.norm(), g.std()) for g in shared_data['gradients'][0]]

In [ ]:
user.plot(true_user_data)

# Reconstruct user data

In [ ]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

In [ ]:
user.plot(reconstructed_user_data, scale=True)

In [ ]:
# How good is the reconstruction?
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

In [ ]:
user.plot(reconstructed_user_data, scale=False)